# A Job Scheduler for Snappy Ubuntu

## Job Sceduler

In [ ]:
import subprocess
import time
import datetime
from pathlib2 import Path

In [ ]:
def read_config(fn = 'config.dat'):
    '''
    Reads the config file, removes comments marked by '#' and replaces multiple whitespaces by single ones.
    '''
    lines = []
    with open(fn) as f:
        for line in f:
            # substring the line to stop before the first occurrence of '#'
            line = line[:line.find('#')]
            # remove whitespaces from the beginning and the end of the line
            line = line.strip()
            # remove all double / multiple spaces by single spaces
            len_old = 0
            while (len(line) != len_old):
                len_old = len(line)
                line = line.replace('  ', ' ')
            # append if remaining line is one or more characters
            if (len(line) > 0):
                lines.append(line)
        return lines

In [ ]:
def get_current_datetime():
    return subprocess.check_output('date +"%Y %m %e %H %M"', shell=True).decode().replace('\n','').split(' ')

In [ ]:
def get_datetime_from_line(line):
    line = line.split(' ')
    return [s for s in line[:5]]

In [ ]:
def get_command_from_line(line):
    line = line.split(' ')
    return ' '.join([str(s) for s in line[5:]])

In [ ]:
def is_equal(dt, dt_):
    return (dt[0] == dt_[0]) and (dt[1] == dt_[1]) and (dt[2] == dt_[2]) and (dt[3] == dt_[3]) and (dt[4] == dt_[4])

In [ ]:
def check_run_condition(dtc, dt, dt_old):
    # do not run twice at the same date and time
    if (is_equal(dt, dt_old)):
        return False
    # replace all '*' with the corresponding value of the current datetime
    for i in range(0, 5):
        if (dtc[i] == '*'):
            dtc[i] = dt[i]
    if (is_equal(dtc, dt)):
        return True
    return False

In [ ]:
lines = read_config()

In [ ]:
lines

['* * * * * /home/eandreas/job_scheduler/test.sh',
 '2022 * * 23 52 java -jar mailalerts_2.0.4.jar']

In [ ]:
dtc = get_datetime_from_line(lines[0])

In [ ]:

dt = get_current_datetime()
time.sleep(60)
dt_ = get_current_datetime()
dt, dt_

(['2022', '12', '17', '22', '59'], ['2022', '12', '17', '23', '00'])

In [ ]:
check_run_condition(dtc, dt, dt_)

True

In [ ]:
check_run_condition(dtc, dt, dt)

False

In [ ]:
dt_ = get_current_datetime()

while True:
    print("Sleeping for 60 seconds ...")
    time.sleep(5)
    dt = get_current_datetime()
    print(f'Current datetime: {dt}')
    lines = read_config()
    for line in lines:
        dtc = get_datetime_from_line(line)
        if check_run_condition(dtc, dt, dt_):
            command = get_command_from_line(line)
            # subprocess.check_output(command, shell=True)
            print(f'Running command: {command}')
    dt_ = dt


## Backup Rotation

### Idea

Keep backups as follows:
* last nd days, nd = 1 meaning keep today and yesterday
* nw = [now, [weekday1, weekday2, ...]] with now the number of weeks to keep and weekday1, ... the weekdays to keep withon the last now weeks
* nm = [nom, [day1, day2, ...]] with nom the number of monts to keep and day1, ... the days to keep within the last nm weeks
* ny = [noy, [month1, month2, ...]] with noy the numeber of years to keepa and month1, ... the month to keep within the last noy years

### Create empty files within directory sim_data to test the deletion

In [ ]:
sim_data = Path('simdata')
Path.mkdir(sim_data, exist_ok=True)

years = [2022]
months28 = [2]
months30 = [4, 6, 9, 11]
months31 = [1, 3, 5, 7, 8, 10, 12]

for y in years:
    for m in months28:
        for d in range(1, 29):
            f = Path(f'{sim_data}/nextcloud_backup_{y}{m:02d}{d:02d}-020000')
            f.touch()
    for m in months30:
        for d in range(1, 31):
            f = Path(f'{sim_data}/nextcloud_backup_{y}{m:02d}{d:02d}-020000')
            f.touch()
    for m in months31:
        for d in range(1, 32):
            f = Path(f'{sim_data}/nextcloud_backup_{y}{m:02d}{d:02d}-020000')
            f.touch()

In [ ]:
dt = subprocess.check_output('date +%Y%m%d-%H%M%S', shell=True).decode().replace('\n', '').split('-')

In [ ]:
dt

['20221217', '230038']

In [ ]:
for fn in sim_data.glob('*[0-9][0-9][0-9][0-9][0-9][0-9]-[0-9][0-9][0-9][0-9][0-9][0-9]*'):
    print(fn)

simdata/nextcloud_backup_20220417-020000
simdata/nextcloud_backup_20221120-020000
simdata/nextcloud_backup_20220517-020000
simdata/nextcloud_backup_20221020-020000
simdata/nextcloud_backup_20220325-020000
simdata/nextcloud_backup_20220225-020000
simdata/nextcloud_backup_20220409-020000
simdata/nextcloud_backup_20220509-020000
simdata/nextcloud_backup_20220329-020000
simdata/nextcloud_backup_20220505-020000
simdata/nextcloud_backup_20220405-020000
simdata/nextcloud_backup_20221004-020000
simdata/nextcloud_backup_20221104-020000
simdata/nextcloud_backup_20220201-020000
simdata/nextcloud_backup_20220301-020000
simdata/nextcloud_backup_20221116-020000
simdata/nextcloud_backup_20220421-020000
simdata/nextcloud_backup_20221016-020000
simdata/nextcloud_backup_20220521-020000
simdata/nextcloud_backup_20220313-020000
simdata/nextcloud_backup_20220213-020000
simdata/nextcloud_backup_20221108-020000
simdata/nextcloud_backup_20221008-020000
simdata/nextcloud_backup_20220622-020000
simdata/nextclou

In [ ]:
d1 = datetime.date(2021, 12, 1)

In [ ]:
now = datetime.date.today()

In [ ]:
now - d1

datetime.timedelta(days=381)

In [ ]:
now + datetime.timedelta(days=3)

datetime.date(2022, 12, 20)